In [1]:
import torch
import time

## 1 Multiple views of a storage

In [2]:
ones = torch.full((13, 13), 1)

In [3]:
ones[ 1, :] = torch.full((1, 13), 2)
ones[ 6, :] = torch.full((1, 13), 2)
ones[11, :] = torch.full((1, 13), 2)

ones[:,  1] = torch.full((1, 13), 2)
ones[:,  6] = torch.full((1, 13), 2)
ones[:, 11] = torch.full((1, 13), 2)

ones[ 3:5,  3:5] = torch.full((2, 2), 3)
ones[ 3:5, 8:10] = torch.full((2, 2), 3)
ones[8:10,  3:5] = torch.full((2, 2), 3)
ones[8:10, 8:10] = torch.full((2, 2), 3)

In [4]:
ones

tensor([[1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 3, 3, 1, 2, 1, 3, 3, 1, 2, 1],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1]])

## 2 Eigendecomposition

In [5]:
seed = 27
torch.random.manual_seed(seed)

In [6]:
M = torch.normal(mean=0.0, std=1.0, size=(20,20))

In [7]:
A = M @ torch.diag(torch.arange(1, 21, dtype=torch.float)) @ torch.inverse(M)

In [8]:
eigval, _ = A.eig()

In [10]:
eigval[:, 0].sort()[0]

tensor([ 1.0000,  2.0000,  3.0000,  4.0000,  5.0000,  6.0000,  7.0000,  8.0000,
         9.0000, 10.0000, 11.0000, 12.0000, 13.0000, 14.0000, 15.0000, 16.0000,
        17.0000, 18.0000, 19.0000, 20.0000])

## 3 Flops per second

In [126]:
I = torch.empty((5000, 5000)).normal_()
J = torch.empty((5000, 5000)).normal_()

In [127]:
start = time.perf_counter()
I @ J 
end = time.perf_counter()

duration = end - start

print(f"Computation time: {duration} seconds.")

Computation time: 27.209303716997965 seconds.


In [83]:
# matrix-matrix product of MxN with NxL has complexity 2MNL - ML FLOPs
# c.f. page 6 of https://mediatum.ub.tum.de/doc/625604/625604
flops = 2 * 5000 ** 3 - 5000 ** 2

In [84]:
# FLOPs per second - should be in the billions or tens of billions
flops / duration

9707102613.518362

## 4 Playing with strides

In [149]:
def mul_row(tensor):
    
    m, n = tensor.size()
    
    result = torch.empty(m, n)
    
    for i in range(m):
        for j in range(n):
            result[i, j] = tensor[i, j] * (i + 1)
            
    return result

In [150]:
mul_row_fast = lambda tensor: torch.mul(torch.arange(1, tensor.size()[0] + 1).view(tensor.size()[0], -1), tensor)

#### Test

In [155]:
def test_speed_mul_row(tensor):
    start_slow = time.perf_counter()
    slow = mul_row(m)
    end_slow = time.perf_counter()

    start_fast = time.perf_counter()
    fast = mul_row_fast(m)
    end_fast = time.perf_counter()

    duration_slow = (end_slow - start_slow) * 1e3
    duration_fast = (end_fast - start_fast) * 1e3

    print(f"Slow: {duration_slow:.3f} ms | Fast: {duration_fast:.3f} ms | Speed-up ratio: {duration_slow / duration_fast:.3f}")
    
#     return duration_slow, duration_fast

In [156]:
m = torch.full((4, 8), 2.0)
M = torch.full((1000, 400), 2.0)

In [157]:
test_speed_mul_row(m)

Slow: 2.095 ms | Fast: 0.284 ms | Speed-up ratio: 7.390


In [158]:
test_speed_mul_row(M)

Slow: 2.882 ms | Fast: 0.172 ms | Speed-up ratio: 16.796
